In [57]:
# Importing necessary modules

import sklearn.datasets as datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn import tree
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [58]:
import os
cwd = os.getcwd()
cwd

'/Users/binshuaiwang/PycharmProjects/XAI_project/XAI_project/Python_Decision_Tree_and_Random_Forest'

In [59]:
import pandas as pd 
# read text file into pandas DataFrame and 
# create header 
import os
cwd = os.getcwd()
mypath = cwd + "/../data/nohold/"
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [60]:
count = 0
for file_name in onlyfiles:
    if "Traj" in file_name:
        if count == 0:
            file_path = cwd + "/../data/nohold/" + file_name
            df = pd.read_csv(file_path, sep=",", header=None)
            df.columns = ["TIME_str", "TIME_sec", "latitude", "longitude", "ALTITUDE_FT", "GROUNDTRACK_DEG", \
                          "GROUNDSPEED_KTS", "VERTICALSPEED_FT_PER_MIN", "Vertical_label","Horizontal_label", "Speed_label"]
            X_df = df[[ "latitude", "longitude", "ALTITUDE_FT", "GROUNDTRACK_DEG", \
                          "GROUNDSPEED_KTS", "VERTICALSPEED_FT_PER_MIN"]]
            Y_target = df["Vertical_label"]
        else:
            file_path = cwd + "/../data/nohold/" + file_name
            df_temp = pd.read_csv(file_path, sep=",", header=None)
            df_temp.columns = ["TIME_str", "TIME_sec", "latitude", "longitude", "ALTITUDE_FT", "GROUNDTRACK_DEG", \
                          "GROUNDSPEED_KTS", "VERTICALSPEED_FT_PER_MIN", "Vertical_label","Horizontal_label", "Speed_label"]
            X_df_temp = df_temp[[ "latitude", "longitude", "ALTITUDE_FT", "GROUNDTRACK_DEG", \
                          "GROUNDSPEED_KTS", "VERTICALSPEED_FT_PER_MIN"]]
            Y_target_temp = df_temp["Vertical_label"]
            X_df = pd.concat([X_df, X_df_temp])
            Y_target = pd.concat([Y_target, Y_target_temp])
        count +=1

In [96]:
target_names =  list(set(list(Y_target)))
word_map_to_num = set(list(Y_target))
print("category:")
word_map_to_num


category:


{'  Over 250 kts under 10000ft',
 ' Climb_Nominal',
 ' Climb_OffNominal',
 ' DefyingCardinalAltitudeRule',
 ' Descend_Nominal',
 ' MaintainCurrentAltitude',
 ' No Conclusion: Vertical'}

In [62]:
n_sample = len(X_df)
print("number of points:", n_sample)
X_df = X_df.head(n_sample)
Y_target = Y_target.head(n_sample)
Y = []
for y in Y_target:
    for idx, s in enumerate(word_map_to_num):
        if s == y:
            Y.append(idx)
Y = np.array(Y)

number of points: 143811


In [93]:
import pandas as pd 
import dtreeviz
from sklearn.tree import export_text
# read text file into pandas DataFrame and 
# create header 
import os
cwd = os.getcwd()

# file_name_test = "Traj_AAL80-11121269.dat" # worst
file_name_test = "Traj_AAL173-11184786.dat" # normal
file_name_test = "Traj_AAL1-10490130.dat" # best

file_path_test = cwd + "/../data/nohold/" + file_name_test


# file_name_test = "Traj_AAL1252-11172342POS_HOLD.dat"

# file_path_test = cwd + "/../data/HOLD/" + file_name_test


df_test = pd.read_csv(file_path_test, sep=",", header=None)
df_test.columns = ["TIME_str", "TIME_sec", "latitude", "longitude", "ALTITUDE_FT", "GROUNDTRACK_DEG", \
              "GROUNDSPEED_KTS", "VERTICALSPEED_FT_PER_MIN", "Vertical_label","Horizontal_label", "Speed_label"]
X_df_test = df_test[[ "latitude", "longitude", "ALTITUDE_FT", "GROUNDTRACK_DEG", \
              "GROUNDSPEED_KTS", "VERTICALSPEED_FT_PER_MIN"]]

In [64]:
print("number of testing points:", len(X_df_test))

number of testing points: 3515


In [65]:
Z_target = df_test["Vertical_label"]
n_sample = len(X_df_test)
X_df_test = X_df_test.head(n_sample)
Z_target = Z_target.head(n_sample)
Z = []
for z in Z_target:
    for idx, s in enumerate(word_map_to_num):
        if s == z:
            Z.append(idx)
Z = np.array(Z)
len(Z)

3515

## target 1: Vertical_label

# Entropy

In [105]:
# Entropy, small tree with train data

criterion = "entropy"
max_depth = 3
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
viz_model = dtreeviz.model(dtree_small_entropy, X_df, Y,
                target_name="target",
                feature_names=list(X_df.columns),
                class_names=target_names)

v = viz_model.view()     # render as SVG into internal object 
v                 # pop up window
v.save("nohold_all"+ "_Vertical_label_" + f"max_depth:{max_depth}"  + ".svg")  # optionally save as svg

0.8823038571458373


/Users/binshuaiwang/PycharmProjects/XAI_project/venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names


In [97]:
# Entropy, small tree with train data

criterion = "entropy"
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names)
print(r)

0.8923239529660457
|--- VERTICALSPEED_FT_PER_MIN <= 160.00
|   |--- VERTICALSPEED_FT_PER_MIN <= -160.00
|   |   |--- ALTITUDE_FT <= 9985.50
|   |   |   |--- GROUNDSPEED_KTS <= 280.50
|   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -608.00
|   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |--- VERTICALSPEED_FT_PER_MIN >  -608.00
|   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |--- GROUNDSPEED_KTS >  280.50
|   |   |   |   |--- GROUNDSPEED_KTS <= 291.50
|   |   |   |   |   |--- class:   Over 250 kts under 10000ft
|   |   |   |   |--- GROUNDSPEED_KTS >  291.50
|   |   |   |   |   |--- class:   Over 250 kts under 10000ft
|   |   |--- ALTITUDE_FT >  9985.50
|   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -864.00
|   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -1440.00
|   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |--- VERTICALSPEED_FT_PER_MIN >  -1440.00
|   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |--- VERTICALSPEED_FT_PER_MIN >

In [91]:
criterion = "entropy"
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=10, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.967255634130908


In [67]:
# Entropy, large tree with train data

criterion = "entropy"
dtree_large_n = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_n.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.9998122535828275


In [68]:
# Entropy, large tree with test data

criterion = "entropy"
dtree_large_entropy = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.9991465149359887


In [69]:
# Entropy, small tree with test data

criterion = "entropy"
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.8711237553342818


In [70]:
criterion = "entropy"
num_of_trees = 3
rf_small_entropy = RandomForestClassifier(n_estimators=num_of_trees)
model=rf_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.9985775248933144


In [89]:
criterion = "entropy"
num_of_trees = 3
rf_small_entropy = RandomForestClassifier(n_estimators=num_of_trees, max_depth=5, min_samples_leaf=20)
model=rf_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.8352773826458038


# Gini

In [72]:
# Gini, small tree with train data

criterion = "gini"
dtree_small_gini = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_gini.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.9024692130643692


In [73]:
# Gini, large tree with train data
criterion = "gini"
dtree_large_gini = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_gini.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.9998122535828275


In [74]:
# Gini, large tree with test data

criterion = "gini"
dtree_large_gini = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_gini.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.9991465149359887


In [75]:
# Gini, small tree with test data

criterion = "gini"
dtree_small_gini = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_gini.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.8597439544807967


## Log_loss

In [76]:
# Log_loss, small tree with train data

criterion = "log_loss"
dtree_small_Log_loss = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.8923239529660457


In [77]:
# Log_loss, large tree with train data
criterion = "log_loss"
dtree_large_Log_loss = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.9998122535828275


In [78]:
# Log_loss, large tree with test data
criterion = "log_loss"
dtree_large_Log_loss = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.9991465149359887


In [79]:
# Log_loss, small tree with test data

criterion = "log_loss"
dtree_small_Log_loss = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.8711237553342818
